# Clustering Assignment 3

## Explore and cluster the neighborhoods in Toronto

#### *Just make sure:*

1. to add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. to generate maps to visualize your neighborhoods and how they cluster together.

In [101]:
#Load the needed data
#load the python environment
import numpy as np
import pandas as pd

data_toronto_location=pd.read_csv('data_tornto_location.csv', index_col=0)

In [102]:
#Show the data
data_toronto_location.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [103]:
#Load the python environment that needed for the following clustering method.
import json
import requests
from pandas.io.json import json_normalize

#load the pltoing environment
import matplotlib.cm as cm
import matplotlib.colors as colors

#k-means
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

**Create a map of the city Toronto**

In [104]:
#Latitude and Longitude of the city Toronto Canada
latitude_to=43.6532
longitude_to=-79.3832

map_toronto=folium.Map(location=[latitude_to,longitude_to], zoom_start=10)

#Add markers to the map
for lat, lng, borough, neighborhood in zip(data_toronto_location['Latitude'], data_toronto_location['Longitude'], data_toronto_location['Borough'], data_toronto_location['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  


map_toronto

### Cluster the each Neighborhood such as an example of "Scarborough"

In [105]:
Scarborough=data_toronto_location[data_toronto_location['Borough']=='Scarborough'].reset_index(drop=True)
Scarborough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Get the Geographical coordinates of "Scarborough"

In [106]:
# create map of toronto
map_Scarborough = folium.Map(location=[latitude_to, longitude_to], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Scarborough['Latitude'], Scarborough['Longitude'], Scarborough['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3126cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

**Cluster all different Borough**

In [107]:
#Different Borough values
data_sort_Borough=data_toronto_location.sort_values(by='Borough',ascending=True)
data_sort_Borough.head()

Borough_uni=data_sort_Borough['Borough'].unique()
Borough_uni_list=list(Borough_uni)
print(Borough_uni_list)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
data_sort_Borough['Borough_num']=lb_make.fit_transform(data_sort_Borough['Borough'])
data_sort_Borough[['Borough','Borough_num']].head()

['Central Toronto', 'Downtown Toronto', 'East Toronto', 'East York', 'Etobicoke', 'Mississauga', 'North York', 'Scarborough', 'West Toronto', 'York']


,Borough,Borough_num
48,Central Toronto,0
63,Central Toronto,0
47,Central Toronto,0
64,Central Toronto,0
65,Central Toronto,0


In [108]:
map_cluster_Borough=folium.Map(location=[latitude_to, longitude_to], zoom_start=10)

#Set color to different value of Borough_uni_list
x = np.arange(len(Borough_uni_list))
ys = [i + x + (i*x)**3 for i in range(len(Borough_uni_list))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data_sort_Borough['Latitude'], data_sort_Borough['Longitude'], data_sort_Borough['Neighborhood'], data_sort_Borough['Borough_num']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_Borough)
       
map_cluster_Borough

***

***

## Like the analysis in Newyork exploration, we here will explore the neighborhood of "Central Toronto"

In [109]:
#Obtain the data
C_toronto=data_sort_Borough[data_sort_Borough['Borough']=='Central Toronto'].reset_index(drop=True)
C_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Borough_num
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,0
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0


In [110]:
from geopy.geocoders import Nominatim

In [111]:
#Get the latitude and longitude location of he central toronto
address='Central Toronto'

geolocatorc_ct = Nominatim(user_agent="CT_explorer")
location_ct = geolocator.geocode(address)
latitude_ct = location.latitude
longitude_ct = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_ct, longitude_ct))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


## Here we will use the Foursquare app

#### Here I need to load my credentials

In [113]:
#CLIENT_ID = 'Your Id'
#CLIENT_SECRET = 'Your secret' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

In [114]:
# Obtain the Neighborhood name, the latitude and lobtitude
Nei_name=C_toronto.loc[0,'Neighborhood']
Nei_lat=C_toronto.loc[0, 'Latitude']
Nei_long=C_toronto.loc[0, 'Longitude']

#Print the First neighborhood
print("Latitude {}, and longitude {} cooresponding to the neighborhood {} in Central Toronto".format(Nei_lat,Nei_long,Nei_name))

Latitude 43.6895743, and longitude -79.38315990000002 cooresponding to the neighborhood Moore Park, Summerhill East in Central Toronto


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [115]:
radius=500
LIMIT=100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,Nei_lat, Nei_long, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=NSSE5SHKHT3CDFYR1BXIMKW0CJYZRDMGEPTU4DMACMSXH2TT&client_secret=WK5RX1PMHRD1BJPBVY2A132YAHMWEK1HK421IBZGLCB21ZDA&ll=43.6895743,-79.38315990000002&v=20180605&radius=500&limit=100'

In [116]:
results=requests.get(url).json()
#results

In [117]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [118]:
#clean the jsan and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Ravine,Trail,43.690356,-79.386841
1,Totum Life Science St. Clair,Gym,43.686525,-79.383449


In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### Explore all the neighborhood in Central Toronto

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
CT_venues = getNearbyVenues(names=C_toronto['Neighborhood'],
                                   latitudes=C_toronto['Latitude'],
                                   longitudes=C_toronto['Longitude']
                                  )


Moore Park, Summerhill East
Roselawn
Davisville
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [122]:
print(CT_venues.shape)
CT_venues.head()

(112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Moore Park, Summerhill East",43.689574,-79.383160,Ravine,43.690356,-79.386841,Trail
1,"Moore Park, Summerhill East",43.689574,-79.383160,Totum Life Science St. Clair,43.686525,-79.383449,Gym
2,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
3,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue
4,Davisville,43.704324,-79.388790,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café


In [123]:
CT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,8,8,8,8,8,8
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
"North Toronto West, Lawrence Park",19,19,19,19,19,19
Roselawn,2,2,2,2,2,2
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",23,23,23,23,23,23


In [124]:
print('There are {} uniques categories.'.format(len(CT_venues['Venue Category'].unique())))

There are 63 uniques categories.


## Analyze Each Neighborhood

In [125]:
# one hot encoding
CT_onehot = pd.get_dummies(CT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CT_onehot['Neighborhood'] = CT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CT_onehot.columns[-1]] + list(CT_onehot.columns[:-1])
CT_onehot = CT_onehot[fixed_columns]

CT_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,"Moore Park, Summerhill East",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [126]:
CT_onehot.shape

(112, 64)

In [127]:
CT_grouped = CT_onehot.groupby('Neighborhood').mean().reset_index()
CT_grouped.head()


,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.0,0.0,0.0,0.0,0.000,0.028571,0.0,0.000000,0.057143,...,0.0,0.057143,0.000000,0.028571,0.028571,0.028571,0.00,0.0,0.0,0.0
1,Davisville North,0.0,0.0,0.0,0.0,0.125,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0
2,"Forest Hill North & West, Forest Hill Road Park",0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.000000,...,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.0
3,Lawrence Park,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.333333,0.000000,...,0.0,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0
4,"Moore Park, Summerhill East",0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.50,0.0,0.0,0.0


In [128]:
CT_grouped.shape

(9, 64)

#### Top 5 venues in each area

In [129]:
num_top_venues = 5

for hood in CT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CT_grouped[CT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1        Dessert Shop  0.09
2      Sandwich Place  0.09
3  Italian Restaurant  0.06
4                 Gym  0.06


----Davisville North----
               venue  freq
0              Hotel  0.12
1   Department Store  0.12
2                Gym  0.12
3               Park  0.12
4  Food & Drink Shop  0.12


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0          Swim School  0.33
1             Bus Line  0.33
2                 Park  0.33
3  American Restaurant  0.00
4  Rental Car Location  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                  Gym   0.5
1                Trail   0.5
2  American Restaurant   0.0
3           Restaurant   0.0
4        Je

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [131]:
#Now let's create the new dataframe and display the top 8 venues for each neighborhood.
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
print(columns)
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CT_grouped['Neighborhood']

for ind in np.arange(CT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

['Neighborhood', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue']


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym,Café
1,Davisville North,Hotel,Food & Drink Shop,Gym,Department Store,Sandwich Place,Park,Dog Run,Breakfast Spot
2,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Trail,Park,Jewelry Store,Flower Shop,Dog Run,Donut Shop,Farmers Market
3,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant
4,"Moore Park, Summerhill East",Gym,Trail,Diner,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [132]:
# set number of clusters
kclusters = 3

CT_grouped_clustering = CT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 1, 0, 0], dtype=int32)

In [133]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels ALL', kmeans.labels_)

CT_merged = C_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CT_merged = CT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CT_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Borough_num,Cluster Labels ALL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,2,Gym,Trail,Diner,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,1,Garden,Music Venue,Yoga Studio,History Museum,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station
2,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym,Café
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,0,Sushi Restaurant,Trail,Park,Jewelry Store,Flower Shop,Dog Run,Donut Shop,Farmers Market
4,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,0,Café,Sandwich Place,Coffee Shop,Pub,Middle Eastern Restaurant,Pharmacy,Pizza Place,Liquor Store


In [135]:
CT_merged.shape

(9, 15)

## Create Maps

In [137]:
# create map
map_clusters_CT = folium.Map(location=[Nei_lat, Nei_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**4 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CT_merged['Latitude'], CT_merged['Longitude'], CT_merged['Neighborhood'], CT_merged['Cluster Labels ALL']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_CT)
       
map_clusters_CT

### Cluster 1

In [136]:
CT_merged.loc[CT_merged['Cluster Labels ALL'] == 0, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Borough_num,Cluster Labels ALL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
2,Central Toronto,0,0,Pizza Place,Sandwich Place,Dessert Shop,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym,Café
3,Central Toronto,0,0,Sushi Restaurant,Trail,Park,Jewelry Store,Flower Shop,Dog Run,Donut Shop,Farmers Market
4,Central Toronto,0,0,Café,Sandwich Place,Coffee Shop,Pub,Middle Eastern Restaurant,Pharmacy,Pizza Place,Liquor Store
5,Central Toronto,0,0,Bus Line,Park,Swim School,Yoga Studio,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant
6,Central Toronto,0,0,Hotel,Food & Drink Shop,Gym,Department Store,Sandwich Place,Park,Dog Run,Breakfast Spot
7,Central Toronto,0,0,Coffee Shop,Clothing Store,Seafood Restaurant,Grocery Store,Fast Food Restaurant,Mexican Restaurant,Diner,Park
8,Central Toronto,0,0,Pub,Coffee Shop,Sports Bar,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store,Pizza Place


### Cluster 2

In [140]:
CT_merged.loc[CT_merged['Cluster Labels ALL'] == 1, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Borough_num,Cluster Labels ALL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
1,Central Toronto,0,1,Garden,Music Venue,Yoga Studio,History Museum,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station


### Cluster 3

In [141]:
CT_merged.loc[CT_merged['Cluster Labels ALL'] == 2, CT_merged.columns[[1] + list(range(5, CT_merged.shape[1]))]]

,Borough,Borough_num,Cluster Labels ALL,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Central Toronto,0,2,Gym,Trail,Diner,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden
